<a href="https://colab.research.google.com/github/Chikuji/AzureML/blob/master/2020_09_23_01_ImplantarAML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Objetivos de aprendizagem

Neste módulo, você vai aprender a:

- Implantar um modelo como um serviço de inferência em tempo real.
-Consumir um serviço de inferência em tempo real.
-Solucionar problemas de implantação de serviço

Como implantar um modelo como um serviço em tempo real

# 1. Registrar um modelo treinado

Depois de treinar um modelo com êxito, você precisará registrá-lo em seu Workspace do Azure Machine Learning. O serviço em tempo real poderá, então, carregar o modelo quando necessário.

In [ ]:
from azureml.core import Model

classification_model = Model.register(workspace=ws,
                       model_name='classification_model',
                       model_path='model.pkl', # local path
                       description='A classification model')

# 2. Definir uma configuração de inferência

O modelo será implantado como um serviço que consiste em:

- Um script para carregar o modelo e retornar previsões para os dados enviados
- Um ambiente no qual o script será executado.

Portanto, você precisará definir o script e o ambiente para o serviço.

Como criar um script de entrada

In [ ]:
import json
import joblib
import numpy as np
from azureml.core.model import Model

# Called when the service is loaded
def init():
    global model
    # Get the path to the registered model file and load it
    model_path = Model.get_model_path('classification_model')
    model = joblib.load(model_path)

# Called when a request is received
def run(raw_data):
    # Get the input data as a numpy array
    data = np.array(json.loads(raw_data)['data'])
    # Get a prediction from the model
    predictions = model.predict(data)
    # Return the predictions as any JSON serializable format
    return predictions.tolist()

Como criar um ambiente

In [ ]:
from azureml.core.conda_dependencies import CondaDependencies

# Add the dependencies for your model
myenv = CondaDependencies()
myenv.add_conda_package("scikit-learn")

# Save the environment config as a .yml file
env_file = 'service_files/env.yml'
with open(env_file,"w") as f:
    f.write(myenv.serialize_to_string())
print("Saved dependency info in", env_file)

Como combinar o script e o ambiente em uma InferenceConfig

In [ ]:
from azureml.core.model import InferenceConfig

classifier_inference_config = InferenceConfig(runtime= "python",
                                              source_directory = 'service_files',
                                              entry_script="score.py",
                                              conda_file="env.yml")

# 3. Definir uma configuração de implantação

Agora que você tem o script de entrada e o ambiente, é necessário configurar a computação na qual o serviço será implantado. Se você estiver fazendo a implantação em um cluster do AKS, precisará criar o cluster e um destino de computação para ele antes da implantação:

In [ ]:
from azureml.core.compute import ComputeTarget, AksCompute

cluster_name = 'aks-cluster'
compute_config = AksCompute.provisioning_configuration(location='eastus')
production_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
production_cluster.wait_for_completion(show_output=True)

Com o destino de computação criado, agora você poderá definir a configuração de implantação, que define a especificação de computação específica do destino para a implantação em contêineres:

In [ ]:
from azureml.core.webservice import AksWebservice

classifier_deploy_config = AksWebservice.deploy_configuration(cpu_cores = 1,
                                                              memory_gb = 1)

# 4. Implantar o modelo

In [ ]:
from azureml.core.model import Model

model = ws.models['classification_model']
service = Model.deploy(workspace=ws,
                       name = 'classifier-service',
                       models = [model],
                       inference_config = classifier_inference_config,
                       deployment_config = classifier_deploy_config,
                       deployment_target = production_cluster)
service.wait_for_deployment(show_output = True)



Para os serviços locais ou da ACI, você poderá omitir o parâmetro deployment_target (ou defini-lo como Nenhum).

Mais informações: Para obter mais informações sobre como implantar modelos com o Azure Machine Learning, confira Implantar modelos com o Azure Machine Learning na documentação.

# Como usar o SDK do Azure Machine Learning

Para teste, use o SDK do Azure Machine Learning para chamar um serviço Web por meio do método run de um objeto WebService que referencia o serviço implantado. Normalmente, você envia dados para o método run no formato JSON com a seguinte estrutura:

In [ ]:
{
  "data":[
      [0.1,2.3,4.1,2.0], // 1st case
      [0.2,1.8,3.9,2.1],  // 2nd case,
      ...
  ]
}

In [ ]:
import json

# An array of new data cases
x_new = [[0.1,2.3,4.1,2.0],
         [0.2,1.8,3.9,2.1]]

# Convert the array to a serializable list in a JSON document
json_data = json.dumps({"data": x_new})

# Call the web service, passing the input data
response = service.run(input_data = json_data)

# Get the predictions
predictions = json.loads(response)

# Print the predicted class for each case.
for i in range(len(x_new)):
    print (x_new[i]), predictions[i] )

# Como usar um ponto de extremidade REST

Em produção, a maioria dos aplicativos cliente não incluirá o SDK do Azure Machine Learning e consumirá o serviço por meio da interface REST. Você pode determinar o ponto de extremidade de um serviço implantado no Azure Machine Learning Studio ou recuperando a propriedade scoring_uri do objeto Webservice no SDK da seguinte maneira:

In [ ]:
endpoint = service.scoring_uri
print(endpoint)

Com o ponto de extremidade conhecido, você pode usar uma solicitação HTTP POST com os dados JSON para chamar o serviço. O seguinte exemplo mostra como fazer isso usando o Python:

In [ ]:
import requests
import json

# An array of new data cases
x_new = [[0.1,2.3,4.1,2.0],
         [0.2,1.8,3.9,2.1]]

# Convert the array to a serializable list in a JSON document
json_data = json.dumps({"data": x_new})

# Set the content type in the request headers
request_headers = { 'Content-Type':'application/json' }

# Call the service
response = requests.post(url = endpoint,
                         data = json_data,
                         headers = request_headers)

# Get the predictions from the JSON response
predictions = json.loads(response.json())

# Print the predicted class for each case.
for i in range(len(x_new)):
    print (x_new[i]), predictions[i] )

# Autenticação

- Chave: as solicitações são autenticadas especificando a chave associada ao serviço.
-Token: as solicitações são autenticadas pelo fornecimento de um JWT (Token Web JSON).

In [ ]:
primary_key, secondary_key = service.get_keys()

In [ ]:
import requests
import json

# An array of new data cases
x_new = [[0.1,2.3,4.1,2.0],
         [0.2,1.8,3.9,2.1]]

# Convert the array to a serializable list in a JSON document
json_data = json.dumps({"data": x_new})

# Set the content type in the request headers
request_headers = { "Content-Type":"application/json",
                    "Authorization":"Bearer " + key_or_token }

# Call the service
response = requests.post(url = endpoint,
                         data = json_data,
                         headers = request_headers)

# Get the predictions from the JSON response
predictions = json.loads(response.json())

# Print the predicted class for each case.
for i in range(len(x_new)):
    print (x_new[i]), predictions[i] )

# Solução de problemas de implantação de serviço

Verificar o estado do serviço

In [ ]:
from azureml.core.webservice import AksWebservice

# Get the deployed service
service = AciWebservice(name='classifier-service', workspace=ws)

# Check its state
print(service.state)

Examinar os logs de serviço

In [ ]:
print(service.get_logs())

In [ ]:
from azureml.core.webservice import LocalWebservice

deployment_config = LocalWebservice.deploy_configuration(port=8890)
service = Model.deploy(ws, 'test-svc', [model], inference_config, deployment_config)

Em seguida, você poderá testar o serviço implantado localmente usando o SDK:

In [ ]:
print(service.run(input_data = json_data))

Em seguida, você poderá solucionar problemas de runtime fazendo alterações no arquivo de pontuação referenciado na configuração de inferência e recarregando o serviço sem reimplantá-lo (algo que só pode ser feito com um serviço local):

In [ ]:
service.reload()
print(service.run(input_data = json_data))

Implantação em um contêiner local